In [25]:
import numpy as np
import copy
import imp
import network
import mynet
imp.reload(network)
imp.reload(mynet)

def printValues(listVals):
   for i in range(len(listVals)):
        print(listVals[i])
        print()

In [26]:
sizes = [4,2,3]
net = network.Network(sizes)

print("original weights: (list of " + str(type(net.weights[0])) + ")")
print(type(net.weights[0]))
printValues(net.weights)
print("shape of net.weights[0]: " + str(net.weights[0].shape))

# manually set weights for connections between layers 0 and 1
net.weights[0]=np.array([[0.1, 0.2, 0.3, 0.4], [1.1,1.2,1.3,1.4]])
# same for layers 1 and 2
net.weights[1]=np.array([[0.5, -0.5], [0.6, -1], [0.7, -2]])

# manually set biases as well
net.biases[0] = np.array([[1.0], [1.0]])
net.biases[1] = np.array([[2.0], [2.0], [2.0]])

print("\nnew weights:")
printValues(net.weights)

print("new biases: (list of " + str(type(net.biases[0])) + ")")
print(type(net.biases[0]))
printValues(net.biases)

# make a copy of this network using my class instead
mine = mynet.Network(sizes)
mine.weights = copy.deepcopy(net.weights)
mine.biases = copy.deepcopy(net.biases)
mine.sizes = copy.deepcopy(net.sizes)

original weights: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[ 0.94001524 -0.66635614 -0.86418007 -2.52057296]
 [ 1.47002361 -1.33013982  1.88398505  0.29164318]]

[[-0.53239085 -1.76236319]
 [-1.71159403  1.63931039]
 [-0.28500115  0.42025313]]

shape of net.weights[0]: (2, 4)

new weights:
[[0.1 0.2 0.3 0.4]
 [1.1 1.2 1.3 1.4]]

[[ 0.5 -0.5]
 [ 0.6 -1. ]
 [ 0.7 -2. ]]

new biases: (list of <class 'numpy.ndarray'>)
<class 'numpy.ndarray'>
[[1.]
 [1.]]

[[2.]
 [2.]
 [2.]]



In [27]:
x = np.array([[1], [2], [3], [4]])
zs, activations = mine.feedforward(x)
print(zs)
print(activations)

[array([[0.],
       [0.],
       [0.],
       [0.]]), array([[ 4.],
       [14.]]), array([[1.99100731],
       [1.58920911],
       [0.68741132]])]
[array([[1],
       [2],
       [3],
       [4]]), array([[0.98201379],
       [0.99999917]]), array([[0.87984967],
       [0.8305048 ],
       [0.66539081]])]


In [28]:
# unit test mine.py vs network.py

# training data lists
X = []
Y = []
X.append(np.array([[1], [2], [3], [4]]))
Y.append(np.array([[0.0], [1.0], [0.0]]))

X.append(np.array([[-1], [-4], [-99], [0.3]]))
Y.append(np.array([[0.333], [0.333], [0.333]]))

X.append(np.array([[7.0], [4], [9], [6]]))
Y.append(np.array([[0], [0], [1.0]]))

# test feedforward function
print("testing forward pass evaluation...")
zs, A = mine.feedforward(X[0])
assert(np.allclose(mine.sigmoid(zs[-1]), A[-1]))
assert(np.allclose(mine.getOutput(X[0]), A[-1]))
assert(np.allclose(net.feedforward(X[0]), mine.getOutput(X[0])))

print("testing backprop function...")
nabla_b, nabla_w = net.backprop(X[0], Y[0])
mine_b, mine_w = mine.backprop(X[0], Y[0])
# https://stackoverflow.com/a/30773738
assert(all([np.allclose(nb, mb) for nb, mb in zip(nabla_b, mine_b)]))
assert(all([np.allclose(nw, mw) for nw, mw in zip(nabla_w, mine_w)]))

print("testing updateMiniBatch function...")
# test with a batch size of 1
index = 0
net.update_mini_batch([(X[index], Y[index])], 0.25)
mine.updateMiniBatch([(X[index], Y[index])], 0.25)
assert(all([np.allclose(b, mb) for b, mb in zip(net.weights, mine.weights)]))
assert(all([np.allclose(w, mw) for w, mw in zip(net.biases, mine.biases)]))

X = X[1:3]
Y = Y[1:3]

batch = [(x,y) for x,y in zip(X,Y)]
iterations = 1000
for i in range(iterations):
    index = i % len(X)
    net.update_mini_batch(batch, 0.25)
    mine.updateMiniBatch(batch, 0.25)
    
# test weights and biases were updated to same values
assert(all([np.allclose(b, mb) for b, mb in zip(net.weights, mine.weights)]))
assert(all([np.allclose(w, mw) for w, mw in zip(net.biases, mine.biases)]))

print("\nfinal outputs on training data:")
# now test:
for x,y in zip(X, Y):
    assert(np.allclose(net.feedforward(x), mine.getOutput(x)))
    print(mine.getOutput(x))
    
print("\nfinished all tests!")

testing forward pass evaluation...
testing backprop function...
testing updateMiniBatch function...

final outputs on training data:
[[0.32729287]
 [0.32836246]
 [0.36324349]]
[[0.03554692]
 [0.03272956]
 [0.92773674]]

finished all tests!
